#### Day 17: Common Table Expressions (CTEs) in SQL 🔗

Today we dive into into CTEs, which allow you to create temporary result sets that simplify complex queries and can even be recursive.

#### 1. Database Connection

In [1]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="30_Days_SQL"
)
cursor = conn.cursor()
print("Connected to '30_Days_SQL'!")

---

#### Practice Exercises 💡

#### 1. Basic CTE: Simplify Sales Summary
Use a CTE to calculate total sales per product and filter those with high sales.

In [2]:
query = """
WITH ProductSales AS (
    SELECT product_id, SUM(order_total) AS total_revenue
    FROM orders
    GROUP BY product_id
)
SELECT p.product_name, ps.total_revenue
FROM ProductSales ps
JOIN products p ON ps.product_id = p.product_id
WHERE ps.total_revenue > 1000
"""
pd.read_sql(query, conn)

#### 2. Multiple CTEs: Modular Querying
Chain CTEs to perform multi-step analysis (filtered orders -> customer counts).

In [3]:
query = """
WITH HighValueOrders AS (
    SELECT order_id, customer_id
    FROM orders
    WHERE order_total > 500
),
CustomerOrderCounts AS (
    SELECT customer_id, COUNT(order_id) AS heavy_order_count
    FROM HighValueOrders
    GROUP BY customer_id
)
SELECT c.customer_name, coc.heavy_order_count
FROM CustomerOrderCounts coc
JOIN customers c ON coc.customer_id = c.customer_id
"""
pd.read_sql(query, conn)

#### 3. Recursive CTE: Organizational Hierarchy
Use a recursive CTE to traverse the manager-employee relationships.

In [4]:
query = """
WITH RECURSIVE EmployeeHierarchy AS (
    -- Anchor: Get top-level managers
    SELECT employee_id, name, manager_id, 0 AS depth
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursion: Join employees with their managers from the previous step
    SELECT e.employee_id, e.name, e.manager_id, eh.depth + 1
    FROM employees e
    INNER JOIN EmployeeHierarchy eh ON e.manager_id = eh.employee_id
)
SELECT * FROM EmployeeHierarchy ORDER BY depth, employee_id
"""
pd.read_sql(query, conn)

In [5]:
conn.close()